In [ ]:
!unzip /content/eyes_open_close.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/open eyes/s0032_02982_0_0_1_0_0_02.png  
  inflating: train/open eyes/s0032_02983_0_0_1_0_0_02.png  
  inflating: train/open eyes/s0032_02984_0_0_1_0_1_02.png  
  inflating: train/open eyes/s0032_02985_0_0_1_0_0_02.png  
  inflating: train/open eyes/s0032_02986_0_0_1_0_1_02.png  
  inflating: train/open eyes/s0032_02987_0_0_1_0_0_02.png  
  inflating: train/open eyes/s0032_02988_0_0_1_0_1_02.png  
  inflating: train/open eyes/s0032_02989_0_0_1_0_0_02.png  
  inflating: train/open eyes/s0032_02990_0_0_1_0_1_02.png  
  inflating: train/open eyes/s0032_02991_0_0_1_0_0_02.png  
  inflating: train/open eyes/s0032_02992_0_0_1_0_1_02.png  
  inflating: train/open eyes/s0032_02993_0_0_1_0_0_02.png  
  inflating: train/open eyes/s0032_02994_0_0_1_0_1_02.png  
  inflating: train/open eyes/s0032_02995_0_0_1_0_0_02.png  
  inflating: train/open eyes/s0032_02996_0_0_1_0_1_02.png  
  inflating: train/open eyes/s0032_02997_0_0_1_0_0_02.png  
  in

In [ ]:
import tensorflow as tf
train_dir ='../content/train'
test_dir  ='../content/test'

In [ ]:
width, height = 86, 86
training=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                          rotation_range=7,
                                                          horizontal_flip=True,
                                                          validation_split=0.05
                                                         ).flow_from_directory(train_dir,
                                                                               class_mode = 'categorical',
                                                                               batch_size = 8,
                                                                               target_size=(width,height),
                                                                               subset="training")
testing=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                         ).flow_from_directory(test_dir,
                                                                               class_mode = 'categorical',
                                                                               batch_size = 8,
                                                                               shuffle = False,
                                                                               target_size=(width,height))
validing=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                          rotation_range=7,
                                                          horizontal_flip=True,
                                                         validation_split=0.05
                                                        ).flow_from_directory(train_dir,
                                                                              batch_size = 8,
                                                                              class_mode = 'categorical',
                                                                              target_size=(width,height),
                                                                              subset='validation',
                                                                              shuffle=True)

Found 56974 images belonging to 2 classes.
Found 24926 images belonging to 2 classes.
Found 2998 images belonging to 2 classes.


In [ ]:
from keras.models import Sequential ,Model
from keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization  ,Activation ,GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import EarlyStopping ,ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
optimizer=Adam(lr=0.001,beta_1=0.9,beta_2=0.99,decay=0.001/32)
EarlyStop=EarlyStopping(patience=10,restore_best_weights=True)
Reduce_LR=ReduceLROnPlateau(monitor='val_accuracy',verbose=2,factor=0.5,min_lr=0.00001)
callback=[EarlyStop , Reduce_LR]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model_res50 = Sequential()

model_res50.add(ResNet50(include_top = False,
                         pooling = 'avg',
                         weights = 'imagenet'))

model_res50.add(Dense(2, activation = 'softmax'))

model_res50.layers[0].trainable = False 

model_res50.summary()

steps_per_epoch_training = len(training)
steps_per_epoch_validation = len(validing)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_4 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
model_res50.compile(optimizer = optimizer, 
                    loss = 'categorical_crossentropy', 
                    metrics = ['accuracy'])

history = model_res50.fit(training,
                          steps_per_epoch = steps_per_epoch_training,
                          validation_steps = steps_per_epoch_validation,
                          epochs = 20,
                          validation_data = validing,
                          verbose = 1,
                          callbacks = callback)

Epoch 1/20
7122/7122 [==============================] - 1887s 260ms/step - loss: 0.6241 - accuracy: 0.6502 - val_loss: 0.6262 - val_accuracy: 0.7382 - lr: 0.0010
Epoch 2/20
7122/7122 [==============================] - 1844s 259ms/step - loss: 0.5812 - accuracy: 0.7247 - val_loss: 0.6227 - val_accuracy: 0.6778 - lr: 0.0010
Epoch 3/20
7122/7122 [==============================] - 2092s 294ms/step - loss: 0.5559 - accuracy: 0.7546 - val_loss: 0.6251 - val_accuracy: 0.6424 - lr: 0.0010


KeyboardInterrupt: ignored

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.title('Training and validation loss')
plt.legend()

plt.show()